# Import libraries


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1. Build the code to scrape the Wikipedia page 


In [2]:
# import required modules 
from bs4 import BeautifulSoup 
import requests

In [3]:
# get URL 
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL) 

# display status code 
print(page.status_code) 
  
## display scrapped data 
#print(page.content) 

# scrape webpage 
soup = BeautifulSoup(page.content, 'html.parser') 
  
# display scrapped data 
#print(soup.prettify())


200


# Build dataframe of the postal code of each neighborhood 
# along with the borough name and neighborhood name.
## Obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe

In [4]:
# create object 
table = soup.find("table") 

List_of_postal_codes_of_Canada = pd.read_html(str(table), flavor='bs4')[0]
#i = 0

#table_of_postal_codes_of_Canada
List_of_postal_codes_of_Canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Alternative:
## Code to create dictionary for postal code list and then Data Frame for the list


dict_of_postal_codes_of_Canada = {}
Postal_Code = []
Borough = []
Neighbourhood = []
for row in table.find_all("tr")[1:]:
    col = row.find_all("td")
    Postal_Code.append(col[0].find(text=True).replace('\n',''))
    Borough.append(col[1].find(text=True).replace('\n',''))
    Neighbourhood.append(col[2].find(text=True).replace('\n',''))   
dict_of_postal_codes_of_Canada["Postal Code"] = Postal_Code
dict_of_postal_codes_of_Canada["Borough"] = Borough
dict_of_postal_codes_of_Canada["Neighbourhood"] = Neighbourhood

List_of_postal_codes_of_Canada = pd.DataFrame(dict_of_postal_codes_of_Canada)

# Clean DataFrame:
## Only leave the cells that have an assigned borough. Remove cells with a borough that is Not assigned

In [5]:
List_of_postal_codes_of_Canada.replace("Not assigned", np.nan, inplace = True)
List_of_postal_codes_of_Canada.dropna(subset = ['Borough'], axis = 0, inplace = True)
List_of_postal_codes_of_Canada.reset_index(drop=True, inplace=True)
List_of_postal_codes_of_Canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Print the number of rows of the dataframe

In [6]:
print("The Shape of the Postal Code Table is:", List_of_postal_codes_of_Canada.shape)

The Shape of the Postal Code Table is: (103, 3)


## Install conda 

In [7]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done

# All requested packages already installed.



# 2. Add geographical coordinates to dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

## Use the csv file to get geographical coordinates of the neighborhoods and to create the location dataframe

In [8]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in my IBM Cloud Object Storage. It includes my credentials.
# REMOVE credentials before share the notebook.


# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

location = pd.read_csv(body)
location.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Add columns with location to dataframe of the postal code 

In [9]:
List_of_postal_codes_of_Canada = pd.merge(List_of_postal_codes_of_Canada, location, on="Postal Code")
List_of_postal_codes_of_Canada.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Explore Neighborhoods in Toronto


## Create DataFrame with list of postal codes of Toronto where the boroughs are contain the word Toronto 

In [10]:
List_of_postal_codes_of_Toronto = List_of_postal_codes_of_Canada[List_of_postal_codes_of_Canada['Borough'].str.contains('Toronto')]

### Definition of the Global variablles  for the API request URL:

In [11]:
# @hidden_cell
# The following code accesses a file in my IBM Cloud Object Storage. It includes my credentials.
# REMOVE credentials before share the notebook.


VERSION = '20180604'
LIMIT = 30
radius = 500


## Create a function to repeat the exploration process to all the neighborhoods in Toronto

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Run the above function on each neighborhood and create a new dataframe called Toronto_venues.

In [15]:
Toronto_venues = getNearbyVenues(names=List_of_postal_codes_of_Toronto['Neighbourhood'],
                                   latitudes=List_of_postal_codes_of_Toronto['Latitude'],
                                   longitudes=List_of_postal_codes_of_Toronto['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

## Let's check the size of the resulting dataframe

In [16]:
print(Toronto_venues.shape)
Toronto_venues.head()

(854, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


## Let's check how many venues were returned for each neighborhood

In [17]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,15,15,15,15,15,15
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


## Let's find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 194 uniques categories.


# 3.1 Analyze Each Neighborhood

In [19]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size

In [20]:
Toronto_onehot.shape

(854, 194)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(20)

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.033333,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
4,Central Bay Street,0.033333,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.033333,0.0,0.000000,0.000000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
6,Church and Wellesley,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.033333,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.066667,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.033333,0.00,0.000000,0.000000,0.0,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000


### Let's confirm the new size

In [22]:
Toronto_grouped.shape

(38, 194)

# Let's put each neighborhood along with the top N most common venues into a _pandas_ dataframe
## First, let's write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Tailor Shop,Museum,Creperie,Basketball Stadium,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Climbing Gym,Bar,Restaurant,Burrito Place,Pet Store,Performing Arts Venue,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Gym / Fitness Center,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Sushi Restaurant,Middle Eastern Restaurant,Park,Comic Shop,Poke Place,Bubble Tea Shop


# 3.2 Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 5 clusters

In [27]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 2, 2, 2, 0, 2, 0], dtype=int32)

## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
#Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged = pd.merge(List_of_postal_codes_of_Toronto, neighborhoods_venues_sorted, on="Neighbourhood")
Toronto_merged.head()

Toronto_merged.tail() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Cocktail Bar,Farmers Market,Café,Seafood Restaurant,Restaurant,Beer Bar,Fountain,Food Truck,Hotel,Concert Hall
34,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Italian Restaurant,Bakery,Café,Restaurant,Jewelry Store,Taiwanese Restaurant,Pub,Butcher,Bank
35,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gluten-free Restaurant,Sandwich Place,Pub,Speakeasy,Bakery,Gym / Fitness Center
36,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Sushi Restaurant,Coffee Shop,Pub,Japanese Restaurant,Juice Bar,Burger Joint,Diner,Dessert Shop,Beer Bar,Indian Restaurant
37,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Light Rail Station,Garden,Gym / Fitness Center,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Butcher


# Finally, let's visualize the resulting clusters

In [31]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

ptyprocess-0.7.0     | 16 KB     | ##################################### | 100% 
libprotobuf-3.15.3   | 2.5 MB    | ##################################### | 100% 
libcblas-3.9.0       | 11 KB     | ##################################### | 100% 
backports.functools_ | 8 KB      | ##################################### | 100% 
tensorboard-1.14.0   | 3.2 MB    | ##################################### | 100% 
protobuf-3.15.3      | 339 KB    | ##################################### | 100% 
_openmp_mutex-4.5    | 5 KB      | ##################################### | 100% 
libedit-3.1.20191231 | 121 KB    | ##################################### | 100% 
libcurl-7.71.1       | 312 KB    | ##################################### | 100% 
libsodium-1.0.18     | 366 KB    | ##################################### | 100% 
jupyterlab_pygments- | 8 KB      | ##################################### | 100% 
cffi-1.14.5          | 225 KB    | ##################################### | 100% 
secretstorage-3.3.1  | 24 KB

dbus-1.13.6          | 585 KB    | ##################################### | 100% 
mkl_random-1.2.0     | 313 KB    | ##################################### | 100% 
networkx-2.5         | 1.2 MB    | ##################################### | 100% 
patsy-0.5.1          | 187 KB    | ##################################### | 100% 
libstdcxx-ng-9.3.0   | 4.0 MB    | ##################################### | 100% 
vincent-0.4.4        | 28 KB     | ##################################### | 100% 
backcall-0.2.0       | 13 KB     | ##################################### | 100% 
brotli-1.0.9         | 389 KB    | ##################################### | 100% 
libevent-2.1.10      | 1.1 MB    | ##################################### | 100% 
termcolor-1.1.0      | 6 KB      | ##################################### | 100% 
mkl-2020.4           | 215.6 MB  | ##################################### | 100% 
brunsli-0.1          | 200 KB    | ##################################### | 100% 
pyasn1-0.4.8         | 53 KB

toml-0.10.2          | 18 KB     | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
six-1.15.0           | 14 KB     | ##################################### | 100% 
werkzeug-1.0.1       | 239 KB    | ##################################### | 100% 
pexpect-4.8.0        | 47 KB     | ##################################### | 100% 
packaging-20.9       | 35 KB     | ##################################### | 100% 
zipp-3.4.0           | 11 KB     | ##################################### | 100% 
_py-xgboost-mutex-2. | 8 KB      | ##################################### | 100% 
zfp-0.5.5            | 190 KB    | ##################################### | 100% 
jmespath-0.10.0      | 21 KB     | ##################################### | 100% 
libiconv-1.16        | 1.4 MB    | ##################################### | 100% 
tornado-6.1          | 646 KB    | ##################################### | 100% 
pytest-6.2.2         | 430 K

done


In [32]:
# create map
map_clusters = folium.Map(location=[Toronto_merged['Latitude'][1], Toronto_merged['Longitude'][1]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
### Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

# Cluster 1: 

In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0,Café,Gastropub,Coffee Shop,Farmers Market,Furniture / Home Store,Japanese Restaurant,Gym,Hotel,Diner,Italian Restaurant
5,Downtown Toronto,0,Farmers Market,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Tailor Shop,Museum,Creperie,Basketball Stadium,Liquor Store
8,Downtown Toronto,0,Coffee Shop,Café,Asian Restaurant,Seafood Restaurant,Jazz Club,Bakery,Speakeasy,Smoke Shop,Restaurant,Plaza
10,Downtown Toronto,0,Hotel,Park,Plaza,Café,Deli / Bodega,Brewery,Ice Cream Shop,Bubble Tea Shop,Lake,Dessert Shop
13,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Steakhouse,Gym,Sandwich Place,Museum,Seafood Restaurant
16,Downtown Toronto,0,Café,Coffee Shop,Gastropub,Hotel,Restaurant,American Restaurant,Japanese Restaurant,Bakery,Gym,Deli / Bodega
20,Central Toronto,0,Hotel,Park,Department Store,Breakfast Spot,Sandwich Place,Food & Drink Shop,Gym / Fitness Center,Cosmetics Shop,Dessert Shop,Deli / Bodega
29,Downtown Toronto,0,Café,Vietnamese Restaurant,Caribbean Restaurant,Mexican Restaurant,Bar,Wine Bar,Organic Grocery,Belgian Restaurant,Record Shop,Cheese Shop
33,Downtown Toronto,0,Cocktail Bar,Farmers Market,Café,Seafood Restaurant,Restaurant,Beer Bar,Fountain,Food Truck,Hotel,Concert Hall
35,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gluten-free Restaurant,Sandwich Place,Pub,Speakeasy,Bakery,Gym / Fitness Center


# Cluster 2

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Bus Line,Swim School,Business Service,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


# Cluster 3: Downtown Toronto

In [41]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
1,Downtown Toronto,2,Coffee Shop,Diner,Sushi Restaurant,Mexican Restaurant,Café,Creperie,Burrito Place,Burger Joint,Park,Bank
2,Downtown Toronto,2,Café,Theater,Modern European Restaurant,Japanese Restaurant,Ramen Restaurant,Sandwich Place,Burger Joint,Shopping Mall,Hotel,Diner
4,East Toronto,2,Health Food Store,Trail,Pub,Convenience Store,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
6,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Sushi Restaurant,Middle Eastern Restaurant,Park,Comic Shop,Poke Place,Bubble Tea Shop
7,Downtown Toronto,2,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Restaurant,Baby Store,Coworking Space
9,West Toronto,2,Bakery,Pharmacy,Music Venue,Grocery Store,Brewery,Bar,Bank,Bus Stop,Supermarket,Pool
11,West Toronto,2,Bar,Asian Restaurant,Men's Store,Wine Bar,Vietnamese Restaurant,Pizza Place,New American Restaurant,Korean Restaurant,Japanese Restaurant,Italian Restaurant
12,East Toronto,2,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Bookstore,Brewery,Restaurant,Bubble Tea Shop,Pub
14,West Toronto,2,Café,Coffee Shop,Breakfast Spot,Climbing Gym,Bar,Restaurant,Burrito Place,Pet Store,Performing Arts Venue,Bakery


# Cluster 4

In [42]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,College Arts Building,Coffee Shop,Discount Store,Diner,Dessert Shop,Department Store
32,Downtown Toronto,3,Park,Trail,Playground,Convenience Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


# Cluster 5

In [43]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Wine Bar,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
